In [1]:
import sys
import os
import glob
home_folder = os.path.dirname(os.getcwd())
sys.path.append(home_folder)

from src.xml_server_connect import *
#from src.xml_report_process import *
from src.utility import *
from src.extract_table import *
from src.pre_process import *
from collections import defaultdict
import pandas as pd
import re
import ftfy
import string
import unicodedata
import calendar
from multiprocessing import Pool
import json
import xml.etree.cElementTree as ET
import lxml.etree
import nltk
from nltk.parse import CoreNLPParser

import unicodedata
import calendar
import datetime

ModuleNotFoundError: No module named 'src'

In [2]:
pos_tagger = CoreNLPParser(url='http://localhost:8080', tagtype='pos')
logging.getLogger().setLevel(logging.INFO)

In [3]:
result_folder = '/mnt/results'
dataset_folder = '/domino/dataset/raw_report' 

In [3]:
import re
growth_risk_list = [
    r'(?:structurally )*weak growths* in (?:\w*? )*advanced(?: and emerging)* econom(?:y|ies)',\
    r'weak(?:er)*(?:[- ]than[- ]expected)* global growths*',\
    r'sharp(?:er)*(?:[- ]than[- ]expected)* global growth (?:slowdown|slow[- ]down)',\
    r'(?:structurally )*weak growths* in (?:\w*? )*emerging(?: and advanced)* (?:econom(?:y|ies)|markets*)',\
    r'slow(?:er)* growths* in (?:\w*? )*advanced(?: and emerging)* econom(?:y|ies)',\
    r'slow(?:er)* growths* in (?:\w*? )*emerging(?: and advanced)* (?:econom(?:y|ies)|markets*)',\
    r'new mediocre', r'U\.*S\.* (?:slowdown|slow[- ]down)', r'China (?:slowdown|slow[- ]down)', \
    r'Europe (?:slowdown|slow[- ]down),'\
    r'sharp growth (?:slowdown|slow[- ]down)(?: and financial risks)* in China', r'financial risks in China',\
    r'(?:slowdown|slow[- ]down) in other large (?:(?:EMs|Emerging Markets*)|frontier econom(?:y|ies)|(?:EMs|Emerging Markets*)/frontier econom(?:y|ies))',\
    r'growth (?:slowdown|slow[- ]down)', \
    r'(?:weak(?:er)*(?:[- ]than[- ]expected)*|slow(?:er)*(?:[- ]than[- ]expected)*) growth', \
    r'excess capacit(?:y|ies)', r'labor shortage',\
]


inflation_risk_list = [
    r'low(?:er)*(?:[- ]than[- ]expected)* inflations*', r'declining inflations*', \
    r'high(?:er)*(?:[- ]than[- ]expected)* inflations*', r'rising inflation',\
]

fiscal_risk_list = [
    r'delays* in fiscal (?:adjustment|reform)', r'fiscal reform (?:fatigue|delays*|standstills*)',\
    r'faltering fiscal reforms*', r'political impasse', r'(?:fiscal|policy) slippage',\
    r'(?:worsen|worsening in) (?:fiscal|debt) sustainabilit(?:y|ies)', r'high(?:er)* debt(?: service)*',\
    r'debt dynamics', r'fiscal discipline',\
    r'stress on sovereigns*', r'sovereign (?:risk|down[- ]*grades*)', r'refinancing risks*', \
    r'roll[- ]*over risks*', r'fiscal pressures*',\
    r'\bSOEs*\b', r'contingent liabilit(?:y|ies)', r'fiscal reform',\
]

structural_reform_risk_list = [
    r'delays* in structural (?:adjustment|reform)', r'structural reform (?:fatigue|delays*|standstills*)',\
    r'faltering structural reforms*', r'structural reforms*', r'structural bottlenecks*',\
]

policy_risk = [
    r'unsustainable macroeconomic polic(?:y|ies)', r'policy uncertaint(?:y|ies)(?: and divergences*)*', \
    r'policy divergences*',\
    r'procyclical polic(?:y|ies)', r'political cycle',\
]

geopolitical_security_risk_list = [
    r'geopolitical (?:uncertaint(?:y|ies)|risks*|tensions*)', r'security (?:uncertaint(?:y|ies)|risks*)',\
    r'political fragmentation',\
    r'risks* of fragmentation(?: and state failure)*', r'security dislocation', r'risks* of state failure',\
    r'political instabilit(?:y|ies)', r'political unrests*', r'political disruptions*', r'socio[- ]*economic disruption',\
    r'social cohesion', r'populism', r'nationalism', r'confidence crisis', r'corruptions*', r'\bconflicts*\b',
    r'political tensions*',\
    r'new governments*', r'planned elections*', \
    r'clear (?:governments* )*majorit(?:y|ies)',\
    r'majority in (?:parliaments*|(?:upper |lower )*houses*|legislatures*|legislative assembl(?:y|ies)|congress(?:es)*|senates*|(?:upper |lower |second )*chambers*(?: of deput(?:y|ies))*|convocations*|diets*|councils*|assembl(?:y|ies))',\
]

trade_risk_list = [
    r'trade conflicts*', r'recovery (?:\w*? )*trading partners*', \
    r'import barriers*', r'import bans*', r'trade tensions*',
    r'(?:slowdown|slow[- ]down) (in growth )*(in|of )*(?:\w*? )*trading partners*',\
    r'rising protections*', r'retreat from multilateralism',\
    r'retreats* (?:from|of) (?:cross[- ]border |global )(?:\w*? )*integrations*',\
    r'retreats* (?:from|of) globali[sz]ations*(?: and international cooperations*)*', \
    r'anti[- ]*globali[sz]ation',\
    r'retreats* (?:from|of) international cooperations*(?: and globali[sz]ations*)*',\
    r'post-Brexit arrangements*', r'leave (the )*(?:EU|European Union)',\
    r'protectionism', r'isolationism', r'disruptions* to trades*', r'diruptions* to global value chains*',\
]

capital_flow_fx_risk_list = [
    r'(?:large|vast|ample|huge|substantial|massive|abundant) (?:energy[- ]related )*fdi (?:in[- ]*|out[- ]*)*flows*',\
    r'capital outflows', r'capital flow volatilit(?:y|ies)', r'capital accounts* pressures*', \
    r'(?:BOP|balance of payments*) pressures*', r'pressures* of (?:BOPs*|balance of payments*|capital accounts*)',\
    r'pressures* on reserves*', r'currenc(?:y|ies) crisis', r'(?:currenc(?:y|ies)|exchange rate|fx) depreciations*',\
    r'exchange rate volatilit(?:y|ies)', \
]

spillover_risk_list = [
    r'spill[- ]*overs*', r'contagion',\
]

global_financial_market_risk_list = [
    r'tight(?:er)* (?:and more volatile )*global financial conditions*',\
    r'tightening (?:of|in) global financial conditions*',\
    r'volatile global financial conditions*', r'surge in global financial volatilit(?:y|ies)', \
    r'side[- ]effects* from global financial conditions*', r'tight(?:er)* U\.*S\.* monetary polic(?:y|ies)',\
    r'strengthening of the U\.*S\.* dollar(?: and/or higher rates*)*', r'strengthening of the U\.*S\.* high(?:er)* rates*',\
    r'surge in the U\.*S\.* dollar', r'dollar strength', \
    r'risk appetites*', r'(?:re-*assessment|repricing) of global risks*',\
]

financial_market_risk_list = [
    r'surge in financial volatilit(?:y|ies)', r'rises* in risk premi(?:um|a)', r'asset prices* decline', \
    r'decompression of credit spreads*', \
    r'tight(?:er)* (?:and more volatile )*financial conditions*',\
    r'tightening (?:of|in) financial conditions*',\
    r'investor (?:confidences*|sentiments*|concerns*)',\
    r'risk aversion', r'flight to safety', r'market concerns', r'confidence shocks*',\
    r'(?:re-*assessment|repricing) of (?:(?:regional )*sovereign )*risks*', \
    r'(?:market|financial) stress(?:es)*',\
    r'borrowing costs*', r'(?:re-*financing|funding) risks*', \
    r'(?:sharp(?:er)*(?:[- ]than[- ]expected)* )*(?:high(?:er)*|rise|rising|increase in) interest rates*',\
    r'(?:protracted|(?:over[- ]*)*long|prolonged|extended|stretched(?: out)|drawn out|long[- ]drawn[- ]out|lengthened|lengthy|dragged out) period of low[- ]interest[- ]rates*',\
    r'financial imbalances* from (?:\w*? )*period of low interest rates*',\
    r'bond market stress(?:es)* from a reassessment in sovereign risks*',\
    r'market down[- ]*turn', r'market volatilit(?:y|ies)', r'volatile market',\
]

financial_system_stability_risk_list = [
    r'financial (?:sector )*(?:in)*stabilit(?:y|ies)', \
    r'systemic risks*', r'financial vulnerabilit(?:y|ies)', \
    r'balance[- ]*sheet risks*', r'sovereign[- ]bank nexus*', \
    r'corporate sector (?:vulnerabilit(?:y|ies)|indebtedness)',\
    r'household sector (?:vulnerabilit(?:y|ies)|indebtedness)',\
    r'high leverage',\
    r'leveraged (?:firms*|households*)',\
    r'bank distress',\
    r'depositor confidence',\
    r'deposit outflow',\
    r'falling net interest margins',\
    r'liquidity risks*',\
    r'(?:ample|vast|immense|tremendous|enormous|massive|plenty of|substantial|enough|sufficient|adequate|abundant|great) liquidit(?:y|ies) in (?:\w*? )*financial markets*',\
    r'Interest rate risks*',\
    r'low[- ]interest[- ]rates',\
    r'(?:increase|rise) in interest rates',\
    r'stretched valuations*',\
    r'decline in (?:asset|house|real estate) prices',\
    r'financial stability risks*',\
    r'NPLs',\
    r'non-*performing loans*',\
]

financial_system_stability_risk_global_list = [
    r'european bank distress',\
    r'inconsistent and partial implementation of financial regulatory reforms*',\
    r'cyber[- ]attacks*',\
    r'reduced financial services by correspondent banks*',\
    r'reduced financial services by global/regional banks',\
    r'de[- ]*risking',\
]

energy_commodity_price_risk_list = [
    r'low(?:er)* oil prices*',r'oil prices* shocks*', r'oil prices* volatilit(?:y|ies)',\
    r'low(?:er) commodity prices*', r'commodity prices* shocks*', r'commodity prices* volatilit(?:y|ies)',\
    r'large swings in energy prices*', r'sizeable deviations from baseline energy prices',\
    r'persistently lower energy prices*', r'sustained decline in energy prices*',\
    r'risks* to energy prices*', r'increased volatilit(?:y|ies) in energy prices',\
    r'low(?:er)* energy prices*',\
]

climate_change_natural_disasters_risk_list = [
    r'climate change', r'earthquakes*', r'hurricanes*', r'floods*', r'draughts*',\
    r'extreme weather', r'global warming', r'temperature', r'sea level', r'natural disasters*',\
]

refugee_risk_list = [
    r'refugee cris[ei]s', r'refugee (?:in[- ]*|out[- ]*)*flows*', r'integration of immigrants*',
]

LEMA_TO_SURFACE = {
    'growth risk':re.compile('|'.join(growth_risk_list), re.I),
    'inflation risk':re.compile('|'.join(inflation_risk_list), re.I),
    'fiscal risk':re.compile('|'.join(fiscal_risk_list), re.I),
    'energy & commodity price shock':re.compile('|'.join(energy_commodity_price_risk_list), re.I),
    'trade & globalization risk':re.compile('|'.join(trade_risk_list), re.I),
    'structural reform risk':re.compile('|'.join(structural_reform_risk_list), re.I),
    'policy risk':re.compile(r'|'.join(policy_risk), re.I),
    'geopolitical and political risk':re.compile(r'|'.join(geopolitical_security_risk_list), re.I),
    'capital flow & fx risk':re.compile(r'|'.join(capital_flow_fx_risk_list), re.I),
    'refugee crisis':re.compile(r'|'.join(refugee_risk_list), re.I),
    'climate change & natural disaster':re.compile(r'|'.join(climate_change_natural_disasters_risk_list), re.I),
    'spillover risk':re.compile(r'|'.join(spillover_risk_list), re.I),
    'global financial market risk':re.compile(r'|'.join(global_financial_market_risk_list), re.I),
    'financial market risk':re.compile(r'|'.join(financial_market_risk_list), re.I),
    'financial system stability risk':re.compile(r'|'.join(financial_system_stability_risk_list), re.I),
    'global financial system stability risk':re.compile(r'|'.join(financial_system_stability_risk_global_list), re.I),
}

In [4]:
LEMA_TO_SURFACE['growth risk']

re.compile(r'(?:structurally )*weak growths* in (?:\w*? )*advanced(?: and emerging)* econom(?:y|ies)|weak(?:er)*(?:[- ]than[- ]expected)* global growths*|sharp(?:er)*(?:[- ]than[- ]expected)* global growth (?:slowdown|slow[- ]down)|(?:structurally )*weak growths* in (?:\w*? )*emerging(?: and advanced)* (?:econom(?:y|ies)|markets*)|slow(?:er)* growths* in (?:\w*? )*advanced(?: and emerging)* econom(?:y|ies)|slow(?:er)* growths* in (?:\w*? )*emerging(?: and advanced)* (?:econom(?:y|ies)|markets*)|new mediocre|U\.*S\.* (?:slowdown|slow[- ]down)|China (?:slowdown|slow[- ]down)|Europe (?:slowdown|slow[- ]down),sharp growth (?:slowdown|slow[- ]down)(?: and financial risks)* in China|financial risks in China|(?:slowdown|slow[- ]down) in other large (?:(?:EMs|Emerging Markets*)|frontier econom(?:y|ies)|(?:EMs|Emerging Markets*)/frontier econom(?:y|ies))|growth (?:slowdown|slow[- ]down)|(?:weak(?:er)*(?:[- ]than[- ]expected)*|slow(?:er)*(?:[- ]than[- ]expected)*) growth|excess capacit(?:y|ies)|

In [5]:
def find_matched_keywords(country, year, table_content):
    record = {} 
    record['country'] = country
    record['year'] = year
    for k,v in LEMA_TO_SURFACE.items():
        record[k] = 0
        if k == 'growth risk':
            sents = nltk.tokenize.sent_tokenize(table_content) 
            for sent in sents:
                if len(sent) < 2: continue
                if re.search(v, table_content) is not None \
                   and re.search('China|U\.*S\.*|Europe|advanced|EM|[Ee]merging', table_content) is not None:
                    if re.search(v, table_content) is not None and re.search('China', table_content) is not None:
                        record['China growth'] = 1
                        record['global growth'] = 1
                    elif re.search(v, table_content) is not None and re.search('U\.*S\.*', table_content) is not None:
                        record['US growth'] = 1
                        record['global growth'] = 1
                    elif re.search(v, table_content) is not None and re.search('Europe', table_content) is not None:
                        record['Europe growth'] = 1
                        record['global growth'] = 1
                    elif re.search(v, table_content) is not None and \
                         re.search('advanced', table_content, re.I) is not None:
                        record['global growth'] = 1
                        record['other advanced economies'] = 1
                    elif re.search(v, table_content) is not None and \
                         re.search('EM|[Ee]merging', table_content) is not None:
                        record['global growth'] = 1
                        record['other EM'] = 1
                else:
                    record['growth risk'] = 1
        elif k == 'geopolitical and political risk':
            sents = nltk.tokenize.sent_tokenize(table_content)
            for sent in sents:
                if len(sent) < 2: continue
                if re.search(v, table_content) is not None \
                   and re.search('middle east|africa|russia|ukraine|geopolitical|regional', table_content, re.I):
                    if re.search(v, table_content) is not None and re.search('middle east', table_content, re.I) is not None:
                        record['global political risk'] = 1
                        record['Middle East risk'] = 1
                    elif re.search(v, table_content) is not None and re.search('africa', table_content, re.I) is not None: 
                        record['global political risk'] = 1
                        record['Africa risk'] = 1
                    elif re.search(v, table_content) is not None and re.search('russia', table_content, re.I) is not None: 
                        record['global political risk'] = 1
                        record['Russia risk'] = 1
                    elif re.search(v, table_content) is not None and re.search('ukraine', table_content, re.I) is not None: 
                        record['global political risk'] = 1
                        record['Ukraine risk'] = 1
                    elif re.search(v, table_content) is not None and re.search('geopolitical|regional', table_content, re.I) is not None: 
                        record['global political risk'] = 1
                        record['other regional risk'] = 1
                else:
                    record[k] = 1
        else:
            if re.search(v, table_content) is not None:
                record[k] = 1
    return record

In [6]:
report_files_list = glob.glob(dataset_folder+'/*.json')
keywords_record = []
for i, report_file in enumerate(report_files_list):
    with open(report_file, 'r', encoding='utf8') as f:
        data = json.load(f)
        country = data['country']
        year = data['year']
        report_xml = data['xml']
    table_list = extract_risk_table_node_from_report(report_xml)
    if len(table_list) == 0: 
        continue
    table_content = extract_table_content(table_list)
    keywords_record.append(find_matched_keywords(country,year,table_content))

In [7]:
keywords_record_df = pd.DataFrame.from_records(keywords_record)
keywords_record_df = keywords_record_df.fillna(0)
keywords_record_df.sort_values(['country', 'year'], inplace=True)
keywords_record_df.set_index(['country', 'year'], inplace=True)
keywords_record_df['count_risk'] = keywords_record_df.sum(axis=1)
keywords_record_df.head(20)

Africa risk  China growth  Europe growth  \
country             year                                             
afghanistan         2014          0.0           0.0            0.0   
                    2015          0.0           1.0            0.0   
                    2017          0.0           1.0            0.0   
albania             2014          0.0           0.0            0.0   
                    2016          0.0           1.0            0.0   
algeria             2013          0.0           0.0            0.0   
                    2014          0.0           0.0            0.0   
                    2016          0.0           0.0            1.0   
                    2017          0.0           0.0            1.0   
                    2018          0.0           0.0            1.0   
antigua and barbuda 2013          0.0           0.0            0.0   
                    2015          0.0           0.0            0.0   
argentina           2016          0.0           1.0            0.0   
                    2017          0.0           1.0            0.0   
armenia             2013          0.0           0.0            0.0   
                    2015          0.0           1.0            0.0   
aruba               2015          0.0           0.0            0.0   
                    2017          0.0           1.0            0.0   
australia           2012          0.0           0.0            0.0   
                    2014          0.0           0.0            0.0   

                          Middle East risk  Russia risk  US growth  \
country             year                                             
afghanistan         2014               0.0          0.0        0.0   
                    2015               1.0          0.0        0.0   
                    2017               0.0          0.0        0.0   
albania             2014               0.0          0.0        0.0   
                    2016               0.0          0.0        0.0   
algeria             2013               0.0          0.0        0.0   
                    2014               0.0          0.0        0.0   
                    2016               1.0          0.0        0.0   
                    2017               1.0          0.0        0.0   
                    2018               1.0          0.0        0.0   
antigua and barbuda 2013               0.0          0.0        0.0   
                    2015               1.0          0.0        0.0   
argentina           2016               0.0          0.0        0.0   
                    2017               0.0          0.0        0.0   
armenia             2013               1.0          0.0        0.0   
                    2015               1.0          0.0        0.0   
aruba               2015               1.0          0.0        1.0   
                    2017               0.0          0.0        0.0   
australia           2012               0.0          0.0        0.0   
                    2014               0.0          0.0        0.0   

                          Ukraine risk  capital flow & fx risk  \
country             year                                         
afghanistan         2014           0.0                       1   
                    2015           0.0                       1   
                    2017           0.0                       1   
albania             2014           0.0                       0   
                    2016           0.0                       0   
algeria             2013           0.0                       0   
                    2014           0.0                       1   
                    2016           0.0                       0   
                    2017           0.0                       0   
                    2018           0.0                       1   
antigua and barbuda 2013           0.0                       0   
                    2015           0.0                       0   
argentina           

In [8]:
summary_year = keywords_record_df.groupby('year').sum()
summary_year

,Africa risk,China growth,Europe growth,Middle East risk,Russia risk,US growth,Ukraine risk,capital flow & fx risk,climate change & natural disaster,energy & commodity price shock,...,inflation risk,other EM,other advanced economies,other regional risk,policy risk,refugee crisis,spillover risk,structural reform risk,trade & globalization risk,count_risk
year,,,,,,,,,,,,,,,,,,,,,
2011,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,...,0,0.0,0.0,0.0,0,0,1,0,0,5.0
2012,2.0,5.0,1.0,4.0,1.0,3.0,0.0,14,1,12,...,1,0.0,0.0,7.0,0,0,24,14,0,272.0
2013,0.0,9.0,8.0,9.0,2.0,4.0,0.0,15,14,30,...,12,3.0,0.0,10.0,5,1,44,45,2,544.0
2014,1.0,38.0,19.0,13.0,21.0,12.0,2.0,28,15,26,...,29,5.0,12.0,12.0,4,1,40,67,1,787.0
2015,1.0,34.0,12.0,30.0,18.0,17.0,0.0,26,20,39,...,40,2.0,27.0,7.0,13,1,49,63,3,819.0
2016,2.0,62.0,9.0,16.0,6.0,21.0,0.0,34,24,57,...,35,1.0,7.0,8.0,9,5,46,81,16,901.0
2017,2.0,46.0,5.0,20.0,1.0,28.0,0.0,47,18,37,...,29,0.0,3.0,20.0,29,5,46,67,42,893.0
2018,3.0,42.0,4.0,19.0,1.0,24.0,0.0,36,20,29,...,36,0.0,1.0,21.0,26,4,56,65,47,841.0


In [9]:
summary_country = keywords_record_df.groupby('country').sum()
summary_country

,Africa risk,China growth,Europe growth,Middle East risk,Russia risk,US growth,Ukraine risk,capital flow & fx risk,climate change & natural disaster,energy & commodity price shock,...,inflation risk,other EM,other advanced economies,other regional risk,policy risk,refugee crisis,spillover risk,structural reform risk,trade & globalization risk,count_risk
country,,,,,,,,,,,,,,,,,,,,,
afghanistan,0.0,2.0,0.0,1.0,0.0,0.0,0.0,3,0,0,...,3,0.0,0.0,1.0,0,0,1,2,0,21.0
albania,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,1,...,0,0.0,0.0,0.0,0,0,1,2,0,13.0
algeria,0.0,0.0,3.0,3.0,0.0,0.0,0.0,2,0,4,...,1,0.0,0.0,1.0,0,0,5,3,0,37.0
antigua and barbuda,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,2,0,...,0,0.0,1.0,0.0,0,0,1,2,0,15.0
argentina,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2,0,0,...,0,0.0,0.0,0.0,0,0,2,2,0,14.0
armenia,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0,0,1,...,1,0.0,0.0,0.0,0,0,2,2,0,16.0
aruba,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0,0,1,...,1,0.0,0.0,0.0,0,0,1,2,0,15.0
australia,0.0,3.0,0.0,0.0,0.0,0.0,0.0,1,0,2,...,0,0.0,0.0,1.0,1,0,2,2,1,36.0
austria,0.0,3.0,0.0,3.0,0.0,1.0,0.0,0,0,1,...,2,0.0,0.0,0.0,0,0,4,4,1,44.0


In [12]:
date = datetime.datetime.now()
date = date.strftime("%Y_%m_%d")
writer = pd.ExcelWriter(path = os.path.join(result_folder,'SPR_risk_analysis_{}.xlsx'.format(date)))
summary_year.to_excel(writer, 'summary_year')
summary_country.to_excel(writer, 'summary_country')
keywords_record_df.to_excel(writer, 'records')